# Keto Diet Minimum Cost

In [1]:
!pip install -r requirements.txt

In [2]:
apikey = "Y9p3guLOgHhSyTUEFI33yP7eJKwQyYxIL9130F7j"

In [3]:
!gpg -d --batch --passphrase "noodle octopus" ../students-9093fa174318.json.gpg > ../students-9093fa174318.json

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [4]:
import fooddatacentral as fdc
import pandas as pd

In [5]:
from eep153_tools import read_sheets
keto = read_sheets("1M_bUswM_uKnkntd3D-OIcm9x_zuQW17oXikwIl8uh8I",
                   sheet="Keto",
                   json_creds='../students-9093fa174318.json')
keto.drop(columns=["Notes"], inplace=True)
keto

,Food,Quantity,Units,Price,Date,Location,FDC
0,Salmon,7.00,oz,6.99,2/27/21,"Safeway, Berkeley",1266522
1,Eggs,21.00,oz,4.99,2/27/21,"Safeway, Berkeley",578432
2,Avocados,24.00,oz,4.36,2/27/21,Berkeley Bowl,493869
3,Chicken breast,3.00,lbs,10.47,2/27/21,"Safeway, Berkeley",1195078
4,Almonds,16.00,oz,9.99,2/27/21,"Safeway, Berkeley",1277992
5,Zucchini,1.00,lbs,3.49,2/27/21,"Safeway, Berkeley",595040
6,Butter,16.00,oz,4.49,2/28/21,"Safeway, Berkeley",1252164
7,Cheese,8.00,oz,2.50,2/28/21,"Safeway, Berkeley",816524
8,Beef,3.50,lbs,19.99,2/28/21,"Safeway, Berkeley",1272030
9,Cauliflower,265.00,g,3.99,2/28/21,"Safeway, Berkeley",1254959


In [6]:
# Convert food quantities to FDC units
keto['FDC Quantity'] = keto[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
keto['FDC Price'] = keto['Price']/keto['FDC Quantity']

keto.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = keto.groupby('Food')['FDC Price'].min()

Prices

Food
Almonds               2.2024179992269266 / hectogram
Avocados              0.6408103087507109 / hectogram
Bacon                 1.7614934748571718 / hectogram
Beef                  1.2591544631644862 / hectogram
Butter                0.9898755572101002 / hectogram
Canned Black Beans    0.2328081488672307 / hectogram
Cashews                3.743449211899221 / hectogram
Cauliflower           1.5056603773584907 / hectogram
Cheese                1.1023113109243878 / hectogram
Chia Seeds            1.4697484145658504 / hectogram
Chicken breast        0.7694132950252227 / hectogram
Edamame               0.8789095519103787 / hectogram
Eggs                  0.8381765244209821 / hectogram
Green Bell Peppers    0.5511556554621939 / hectogram
Liver (pork)          0.6613867865546326 / hectogram
Peanut Butter          1.173839067099926 / hectogram
Peas                  0.5511556554621939 / hectogram
Plain Greek Yogurt    0.6602844752437083 / hectogram
Salmon                3.5223570575366723 

In [8]:
keto_nutrients = {}
count = 0
for food in  keto.Food.tolist():
    try:
        FDC = keto.loc[keto.Food==food,:].FDC[count]
        count+=1
        keto_nutrients[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

keto_nutrients = pd.DataFrame(keto_nutrients,dtype=float)

keto_nutrients.fillna(0)
keto_nutrients

,Salmon,Eggs,Avocados,Chicken breast,Almonds,Zucchini,Butter,Cheese,Beef,Cauliflower,...,Bacon,Green Bell Peppers,Chia Seeds,Tomatoes,Tofu,Edamame,Cashews,Walnuts,Peas,Canned Black Beans
10:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.000
12:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.000
14:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.000
16:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.05,NaN,0.020,NaN,NaN,NaN,NaN,NaN,0.636
16:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.001,NaN,NaN,NaN,NaN,NaN,0.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Vitamin E, added",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.000
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.40,NaN,7.900,NaN,NaN,NaN,NaN,NaN,9.700
Vitamins and Other Components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,0.000,NaN,NaN,NaN,NaN,NaN,0.000
Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,93.89,NaN,94.520,NaN,NaN,NaN,NaN,NaN,60.010


## Keto Diet Constraints

Because keto diet is constrained by certain nutrients, namely carbohydrates and protein, we created our own diet_minimum and maximum tables to import.

In [9]:
# reading keto diet min table from google sheets
keto_min = read_sheets("1eWvcfjdO1hW4gm1XF9SMtJ7KPi95sZAbvt6nSX5cnnc",
                   sheet="diet_minimums",
                   json_creds='../students-9093fa174318.json')

keto_min = keto_min.set_index('Nutrition').drop('Source', axis = 1)
keto_min= keto_min.drop(['Carbohydrate, by difference', 'Protein'], axis = 0)

In [10]:
keto_min

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,25.0,25.0,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,100.0,120.0,18.0,8.0,8.0,8.0
Total lipid (fat),72.0,86.4,100.8,115.2,129.6,129.6,158.4,144.0,173.0,129.6,158.4,115.2,144.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0
"Phosphorus, P",460.0,500.0,500.0,1250.0,1250.0,1250.0,1250.0,700.0,700.0,700.0,700.0,700.0,700.0


In [11]:
# reading keto max table from google sheets
keto_max = read_sheets("1eWvcfjdO1hW4gm1XF9SMtJ7KPi95sZAbvt6nSX5cnnc",
                   sheet="diet_maximums",
                   json_creds='../students-9093fa174318.json')

keto_max = keto_max.set_index('Nutrition').drop('Source', axis = 1)

In [12]:
# we have to do this otherwise the keto diet problem is infeasible
keto_max.loc['Protein'] = 208

In [13]:
keto_max

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Carbohydrate, by difference",50,50,50,50,50,50,50,50,50,50,50,50,50
Protein,208,208,208,208,208,208,208,208,208,208,208,208,208


In [14]:
from  scipy.optimize import linprog as lp
import numpy as np
def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.

    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(diet_min.index)]

    Amax = Aall.loc[Aall.index.intersection(diet_max.index)]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,-Amax])

    b = pd.concat([diet_min,-diet_max]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight

    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b

    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

### Keto Diet For Females 19-30

In [15]:
group = "F 19-30"

In [16]:
result_fem = solve_subsistence_problem(keto_nutrients, Prices, keto_min[group], keto_max[group], tol=1e-6)

In [17]:
result_fem

       A: Food                            Spinach  Almonds  Zucchini  Chia Seeds  \
Nutrition                                                                
Energy                            24.00   613.00     21.00      556.00   
Fiber, total dietary               2.40     9.70      1.10       44.40   
Folate, DFE                        0.00     0.00      0.00        0.00   
Calcium, Ca                       94.00   258.00     21.00        0.00   
Iron, Fe                           3.18     3.48      0.44        0.00   
Total lipid (fat)                  0.00    54.84      0.00       22.22   
Magnesium, Mg                     71.00     0.00      0.00        0.00   
Niacin                             0.00     0.00      0.00        0.00   
Phosphorus, P                      0.00     0.00      0.00        0.00   
Potassium, K                     553.00     0.00    222.00        0.00   
Riboflavin                         0.20     0.00      0.00        0.00   
Thiamin                     

In [18]:
diet = result_fem.diet

print("Diet (in 100s of grams or milliliters):")
print(diet[diet >= 1e-6])

Diet (in 100s of grams or milliliters):
Food
Spinach               0.183995
Peanut Butter         1.625995
Liver (pork)          4.131735
Avocados              5.909060
Canned Black Beans    0.123151
Cheese                1.204044
dtype: float64


In [19]:
print("Cost of diet for %s is $%4.2f per day." % (group,result_fem.fun))

Cost of diet for F 19-30 is $9.99 per day.


In [20]:
tab = pd.DataFrame({"Outcome": np.abs(result_fem.A.dot(diet)), "Recommendation": np.abs(result_fem.b)})
print("With the following nutritional outcomes of interest:")
tab.dropna(how = 'all')

With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,4922.687534,2000.0
"Fiber, total dietary",54.953005,25.0
"Folate, DFE",1025.861677,400.0
"Calcium, Ca",1000.000003,1000.0
"Iron, Fe",100.000000,100.0
Total lipid (fat),228.477528,144.0
"Magnesium, Mg",368.385511,310.0
Niacin,84.948107,14.0
"Phosphorus, P",1759.993863,700.0


### Keto Diet For Males 19-30

In [21]:
group2 = 'M 19-30'

In [22]:
result_male = solve_subsistence_problem(keto_nutrients, Prices, keto_min[group2], keto_max[group2], tol=1e-6)

In [23]:
result_male

       A: Food                            Spinach  Almonds  Zucchini  Chia Seeds  \
Nutrition                                                                
Energy                            24.00   613.00     21.00      556.00   
Fiber, total dietary               2.40     9.70      1.10       44.40   
Folate, DFE                        0.00     0.00      0.00        0.00   
Calcium, Ca                       94.00   258.00     21.00        0.00   
Iron, Fe                           3.18     3.48      0.44        0.00   
Total lipid (fat)                  0.00    54.84      0.00       22.22   
Magnesium, Mg                     71.00     0.00      0.00        0.00   
Niacin                             0.00     0.00      0.00        0.00   
Phosphorus, P                      0.00     0.00      0.00        0.00   
Potassium, K                     553.00     0.00    222.00        0.00   
Riboflavin                         0.20     0.00      0.00        0.00   
Thiamin                     

In [24]:
diet = result_male.diet

print("Diet (in 100s of grams or milliliters):")
print(diet[diet >= 1e-6])

Diet (in 100s of grams or milliliters):
Food
Spinach          0.550889
Eggs             4.225534
Peanut Butter    1.646542
Liver (pork)     4.590068
Avocados         3.854601
Cheese           0.825764
dtype: float64


In [25]:
print("Cost of diet for %s is $%4.2f per day." % (group2,result_male.fun))

Cost of diet for M 19-30 is $12.50 per day.


In [26]:
tab2 = pd.DataFrame({"Outcome": np.abs(result_male.A).dot(diet), "Recommendation": np.abs(result_male.b)})
print("With the following nutritional outcomes of interest:")
tab2

With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,5227.418750,2400.0
"Fiber, total dietary",38.905966,25.0
"Folate, DFE",1114.697060,400.0
"Calcium, Ca",1000.000002,1000.0
"Iron, Fe",120.000000,120.0
Total lipid (fat),228.971225,173.0
"Magnesium, Mg",400.000001,400.0
Niacin,92.131643,16.0
"Phosphorus, P",1880.117424,700.0


### Total Berkeley Keto Diet Cost

In [27]:
prop_keto_us = .06

# There are 41910 students at UC Berkeley
berkeley_ketos = 41910 * prop_keto_us
berkeley_ketos_cost =  round(.5 * (berkeley_ketos * result_fem.fun) + .5 * (berkeley_ketos * result_male.fun),2)
print('The total price of berkeley mininum cost diet is $' + str(berkeley_ketos_cost))

The total price of berkeley mininum cost diet is $28269.81
